## TensorFlow中的深度神经网络
### 代码
#### TensorFlow MNIST

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot = True, reshape = False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


使用TensorFlow提供的MNIST数据集，它把分批和独热码都处理好了。  
### 学习参数 Learning Parameters

In [5]:
import tensorflow as tf  
## 参数 Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128 # 如果没有足够内存，可以降低batch size
display_step = 1

n_input = 784 # MNIST data input(img shape: 28*28)
n_classes = 10 # MNIST total classes(0-9 digits)

### 隐藏层参数 Hidden Layer Parameters

In [6]:
n_hidden_layer = 256 # 特征的层数

n_hidden_layer 决定了神经网络隐藏层的大小，也被称作层的宽度。   
### 权重和偏置项 

In [12]:
# Store layers weight and bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input,n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

深层神经网络有多个层，每个层有自己的权重和偏置项。'hidden_layer'的权重和偏置项只属于隐藏层，'out'的权重和偏置项只属于输出层。如果神经网络比这更深，那每一层都有权重和偏置项。
### 输入 Input

In [13]:
# tf Graph input
x = tf.placeholder('float',[None,28,28,1])
y = tf.placeholder('float',[None, n_classes])   

x_flat = tf.reshape(x,[-1, n_input])

MNIST 数据集是由 28px *28px单通道图片组成。tf.reshape()函数把 28px *28px的矩阵转换为784px*1px的单行向量x。
### 多层感知器 Multilayer Perceptron

In [15]:
# ReLU 作为隐藏层激活函数
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']),biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

#输出层的线性激活函数
logits = tf.add(tf.matmul(layer_1, weights['out']),biases['out'])

tf.add(tf.matmul(x_flat, weights['hidden_layer']),biases['hidden_layer']),就是xW+b。把线性函数与ReLU组合在一起，形成一个2层网络。
### 优化器 Optimizer

In [16]:
# 定义误差值和优化器

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, 
                                                              labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

### Session

In [18]:
# 初始化变量
init = tf.global_variables_initializer()

# 启动图
with tf.Session() as sess:
    sess.run(init)
    # 训练循环
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # 遍历所有batch
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # 运行优化器进行反向传导，计算cost（获取loss的值）
            sess.run(optimizer, feed_dict = {x: batch_x, y: batch_y})

TensorFlow中的MNIST库提供了分批接收数据的能力。调用mnist.train.next_batch()函数返回训练数据的一个子集。  
## 保存和读取 TensorFlow 模型  
训练一个模型的时间很长。但是你一旦关闭了TensorFlow session, 你所有训练的权重和偏置项都丢失了。如果你计划在之后重新使用这个模型，你需要重新训练！  
幸运的是， TensorFlow 可以让你通过一个叫 tf.train.Saver的类把你的进程保存下来，这个累可以把任何 tf.Variable 存到你的文件系统。  
### 保存变量  
让我们通过一个简单的例子来保存 weights和 bias Tensors。 第一个例子你只是存两个变量，后面会教你如何把一个实际模型的所有权重保存下来。  

In [32]:
import tensorflow as tf 

# 文件保存路径
save_file = '.model/ckpt'

# 两个Tensor 变量： 权重和偏置项
weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

# 用来存取 Tensor 变量的类
saver = tf.train.Saver()

with tf.Session() as sess:
    # 初始化所有变量
    sess.run(tf.global_variables_initializer())
    
    # 显示变量和权重
    print('weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))
    
    # 保存模型
    saver.save(sess, save_file)

weights:
[[-0.35832396 -1.99825037  0.14570355]
 [-1.41003752 -1.92391455 -1.036587  ]]
Bias:
[ 0.12552246 -1.36862433  0.75178486]


weights 和 bias Tensors 用tf.truncated_normal()函数设定了随机值。用tf.train.Saver.save()函数把这些值被保存在save_file位置，命名为'model.ckpt',('.ckpt'扩展名表示'checkpoint')。  
## 加载变量  
现在这些变量已经存好了，让我们把它们加载到新模型里。  

In [33]:
# Remove the previous weights and bias
# 移除之前的权重和偏置项  
tf.reset_default_graph()

# 两个变量： 权重和偏置项  
weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

# 用来存取Tensor变量的类
saver = tf.train.Saver()

with tf.Session() as sess:
    # 加载权重和偏置项
    saver.restore(sess, save_file)
    
    # 显示权重和偏置项
    print('weight:')
    print(sess.run(weights))
    print('bias:')
    print(sess.run(bias))

INFO:tensorflow:Restoring parameters from .model/ckpt
weight:
[[ 0.15636076 -0.66142511 -0.45238689]
 [-0.92793959 -0.45580754  0.11625437]]
bias:
[-0.28037843  0.49187064 -0.09098915]


### 保存一个训练好的模型

In [37]:
# Remove previous Tensors and Operations
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

learning_rate = 0.001
n_input = 784
n_classes = 10

# Import MNIST data
mnist = input_data.read_data_sets('.', one_hot = True)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# weights and bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# logits - xW+b
logits = tf.add(tf.matmul(features, weights),bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels = labels ))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# calcuate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


### 让我们训练模型并保存权重

In [ ]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

# Launch the graph
# 启动图
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 训练循环
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        